# 年限別市中残存額分析

選択した日付時点での年限別（残存期間別）の市中残存額を可視化します。

## 計算方法
- **市中残存額** = 累積発行額 - 日銀保有額
- **残存期間** = (償還日 - 選択日付) ÷ 365.25
- **年限グリッド**: 1年刻みで集計（調整可能）

## データソース
- `bond_data`: 基本情報（銘柄コード、償還日）
- `bond_auction`: 発行額（total_amount）
- `boj_holdings`: 日銀保有額（face_value）

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
from supabase import create_client

# 日本語フォント設定
plt.rcParams['font.family'] = 'Hiragino Sans'
plt.rcParams['axes.unicode_minus'] = False

# Supabase接続
load_dotenv()
url = os.getenv('SUPABASE_URL')
key = os.getenv('SUPABASE_KEY')
supabase = create_client(url, key)

print('Supabase接続完了')

---
## 📌 日付選択設定

以下のセルで日付を変更すると、すべての計算・グラフに反映されます。

In [ ]:
# ========================================
# 📌 日付選択設定（ここを変更してください）
# ========================================

# 分析対象日付を指定（YYYY-MM-DD形式）
SELECTED_DATE = '2024-01-31'  # デフォルト: 2024年1月31日

print(f'選択された日付: {SELECTED_DATE}')

## データ取得関数

In [ ]:
def get_all_bonds_with_maturity(selected_date: str) -> pd.DataFrame:
    """
    全銘柄の基本情報を取得し、残存期間を計算

    Returns:
        DataFrame with columns: bond_code, due_date, maturity_years
    """
    # bond_dataから全銘柄の情報を取得
    result = supabase.table('bond_data') \
        .select('bond_code, due_date') \
        .not_.is_('due_date', 'null') \
        .execute()

    if not result.data:
        return pd.DataFrame()

    df = pd.DataFrame(result.data)
    df['due_date'] = pd.to_datetime(df['due_date'])

    # 重複除去（銘柄コードでユニーク）
    df = df.drop_duplicates(subset='bond_code')

    # 残存期間を計算
    selected_dt = datetime.strptime(selected_date, '%Y-%m-%d')
    df['maturity_years'] = (df['due_date'] - selected_dt).dt.days / 365.25

    # 負の値（既に満期）を除外
    df = df[df['maturity_years'] > 0]

    # 40年超を除外
    df = df[df['maturity_years'] <= 40]

    return df


def get_auction_history(bond_code: str) -> pd.DataFrame:
    """指定銘柄の発行入札履歴を取得"""
    result = supabase.table('bond_auction') \
        .select('auction_date, total_amount') \
        .eq('bond_code', bond_code) \
        .order('auction_date') \
        .execute()

    if not result.data:
        return pd.DataFrame()

    df = pd.DataFrame(result.data)
    df['auction_date'] = pd.to_datetime(df['auction_date'])
    df['total_amount'] = pd.to_numeric(df['total_amount'], errors='coerce')
    return df


def get_boj_history(bond_code: str) -> pd.DataFrame:
    """指定銘柄の日銀保有履歴を取得"""
    result = supabase.table('boj_holdings') \
        .select('data_date, face_value') \
        .eq('bond_code', bond_code) \
        .order('data_date') \
        .execute()

    if not result.data:
        return pd.DataFrame()

    df = pd.DataFrame(result.data)
    df['data_date'] = pd.to_datetime(df['data_date'])
    df['face_value'] = pd.to_numeric(df['face_value'], errors='coerce')
    return df


print('データ取得関数を定義しました')

## 市中残存額計算関数

In [ ]:
def calculate_cumulative_issuance(auction_history: pd.DataFrame,
                                 target_date: str) -> float:
    """
    指定日までの累積発行額を計算

    Returns:
        累積発行額（億円）、発行前の場合はNone
    """
    target_dt = datetime.strptime(target_date, '%Y-%m-%d')

    # 対象日以前の発行データのみ抽出
    past_auctions = auction_history[auction_history['auction_date'] <= target_dt]

    if past_auctions.empty:
        return None  # 発行前

    return past_auctions['total_amount'].sum()


def get_latest_boj_holding(boj_history: pd.DataFrame,
                           target_date: str) -> float:
    """
    指定日時点での最新の日銀保有額を取得

    Returns:
        日銀保有額（億円）、データなしの場合はNone
    """
    target_dt = datetime.strptime(target_date, '%Y-%m-%d')

    # 対象日以前のデータのみ抽出
    past_holdings = boj_history[boj_history['data_date'] <= target_dt]

    if past_holdings.empty:
        return None

    # 最新の値を取得
    return past_holdings.iloc[-1]['face_value']


def calculate_market_amount_for_bond(bond_code: str,
                                     target_date: str) -> float:
    """
    指定日の指定銘柄の市中残存額を計算

    Returns:
        市中残存額（億円）、計算不可の場合は0
    """
    # データ取得
    auction_history = get_auction_history(bond_code)
    boj_history = get_boj_history(bond_code)

    # 累積発行額を計算
    cumulative_issuance = calculate_cumulative_issuance(auction_history, target_date)

    if cumulative_issuance is None:
        return 0  # 発行前

    # 日銀保有額を取得
    boj_holding = get_latest_boj_holding(boj_history, target_date)

    # 市中残存額を計算
    if boj_holding is not None:
        market_amount = cumulative_issuance - boj_holding
    else:
        # 日銀データなし → 全額が市中に存在
        market_amount = cumulative_issuance

    return max(market_amount, 0)  # 負の値を0に


print('市中残存額計算関数を定義しました')

## 年限グリッド集計関数

In [ ]:
def calculate_market_amount_by_maturity(selected_date: str,
                                       bucket_size: float = 1.0) -> pd.DataFrame:
    """
    年限別の市中残存額を計算

    Args:
        selected_date: 基準日（YYYY-MM-DD）
        bucket_size: グリッド幅（年）

    Returns:
        DataFrame with columns: maturity_bucket, market_amount
    """
    # 全銘柄の基本情報を取得
    bonds_df = get_all_bonds_with_maturity(selected_date)

    if bonds_df.empty:
        print('対象銘柄がありません')
        return pd.DataFrame()

    print(f'対象銘柄数: {len(bonds_df)}')

    # 各銘柄の市中残存額を計算
    market_amounts = []

    for idx, row in bonds_df.iterrows():
        bond_code = row['bond_code']
        market_amount = calculate_market_amount_for_bond(bond_code, selected_date)

        if market_amount > 0:
            market_amounts.append({
                'bond_code': bond_code,
                'maturity_years': row['maturity_years'],
                'market_amount': market_amount
            })

        # 進捗表示（100件ごと）
        if (idx + 1) % 100 == 0:
            print(f'処理中... {idx + 1}/{len(bonds_df)}')

    result_df = pd.DataFrame(market_amounts)

    if result_df.empty:
        print('市中残存額データがありません')
        return pd.DataFrame()

    # 年限バケットを作成
    result_df['maturity_bucket'] = (result_df['maturity_years'] // bucket_size) * bucket_size

    # 年限別に集計
    summary_df = result_df.groupby('maturity_bucket')['market_amount'].sum().reset_index()
    
    # ★ 全年限バケット（0-40年）を作成して欠損を0で埋める
    all_buckets = pd.DataFrame({
        'maturity_bucket': np.arange(0, 40, bucket_size)
    })
    
    # マージして欠損値を0で埋める
    summary_df = all_buckets.merge(summary_df, on='maturity_bucket', how='left')
    summary_df['market_amount'] = summary_df['market_amount'].fillna(0)
    summary_df = summary_df.sort_values('maturity_bucket')

    # バケットラベルを作成
    summary_df['bucket_label'] = summary_df['maturity_bucket'].apply(
        lambda x: f'{int(x)}-{int(x + bucket_size)}年'
    )

    return summary_df


print('年限グリッド集計関数を定義しました')

## 可視化関数

In [ ]:
def plot_market_amount_by_maturity(data: pd.DataFrame,
                                   selected_date: str,
                                   figsize=(16, 8)):
    """
    年限別市中残存額を棒グラフで表示
    """
    if data.empty:
        print('表示するデータがありません')
        return

    fig, ax = plt.subplots(figsize=figsize)

    # 棒グラフ
    bars = ax.bar(data['maturity_bucket'],
                  data['market_amount'],
                  width=0.8,
                  color='steelblue',
                  edgecolor='navy',
                  alpha=0.7)

    # タイトル・ラベル
    ax.set_title(f'年限別市中残存額（{selected_date} 時点）',
                 fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('残存期間（年）', fontsize=13)
    ax.set_ylabel('市中残存額（億円）', fontsize=13)

    # X軸の目盛りとラベル
    ax.set_xticks(data['maturity_bucket'])
    ax.set_xticklabels(data['bucket_label'], rotation=45, ha='right')

    # グリッド
    ax.grid(True, axis='y', alpha=0.3, linestyle='--')
    ax.set_axisbelow(True)

    # 統計情報を表示
    total_amount = data['market_amount'].sum()
    stats_text = f"合計市中残存額: {total_amount:,.0f}億円\n"
    stats_text += f"対象年限範囲: {data['maturity_bucket'].min():.0f}-{data['maturity_bucket'].max():.0f}年\n"
    stats_text += f"バケット数: {len(data)}"

    ax.text(0.98, 0.97, stats_text,
            transform=ax.transAxes,
            fontsize=11,
            verticalalignment='top',
            horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

    plt.tight_layout()
    plt.show()

    # データ表示
    print('\n【年限別集計結果】')
    print(data[['bucket_label', 'market_amount']].to_string(index=False))
    print(f'\n合計: {total_amount:,.0f}億円')


print('可視化関数を定義しました')

---
## 実行例

In [ ]:
# 年限別市中残存額を計算
print(f'計算開始: {SELECTED_DATE}')
maturity_data = calculate_market_amount_by_maturity(
    selected_date=SELECTED_DATE,
    bucket_size=1.0  # 1年刻み
)

# グラフ表示
plot_market_amount_by_maturity(maturity_data, SELECTED_DATE)

## グリッド幅カスタマイズ例（オプション）

In [ ]:
# 異なるグリッド幅で試す場合
# bucket_size = 2.0  # 2年刻み
# maturity_data_2y = calculate_market_amount_by_maturity(SELECTED_DATE, bucket_size=2.0)
# plot_market_amount_by_maturity(maturity_data_2y, SELECTED_DATE)

## 2つの日付の比較（1年間の変動確認）

In [ ]:
# 2つの日付を比較
DATE_1 = '2025-11-15'  # 2025年11月
DATE_2 = '2024-11-15'  # 1年前

print(f'比較対象日付:')
print(f'  日付1: {DATE_1}')
print(f'  日付2: {DATE_2}')
print()

# それぞれの日付でデータ計算
print('=' * 60)
print(f'【{DATE_1} のデータ計算】')
print('=' * 60)
data_1 = calculate_market_amount_by_maturity(DATE_1, bucket_size=1.0)

print()
print('=' * 60)
print(f'【{DATE_2} のデータ計算】')
print('=' * 60)
data_2 = calculate_market_amount_by_maturity(DATE_2, bucket_size=1.0)

In [ ]:
# 2つの日付を並べて比較表示
def plot_comparison(data_1: pd.DataFrame, date_1: str,
                   data_2: pd.DataFrame, date_2: str,
                   figsize=(18, 6)):
    """
    2つの日付の年限別市中残存額を並べて比較表示
    """
    if data_1.empty or data_2.empty:
        print('表示するデータがありません')
        return
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # グラフ1: 日付1
    ax1.bar(data_1['maturity_bucket'],
            data_1['market_amount'],
            width=0.8,
            color='steelblue',
            edgecolor='navy',
            alpha=0.7)
    
    ax1.set_title(f'年限別市中残存額\n{date_1} 時点',
                  fontsize=14, fontweight='bold', pad=15)
    ax1.set_xlabel('残存期間（年）', fontsize=11)
    ax1.set_ylabel('市中残存額（億円）', fontsize=11)
    ax1.set_xticks(data_1['maturity_bucket'])
    ax1.set_xticklabels(data_1['bucket_label'], rotation=45, ha='right', fontsize=8)
    ax1.grid(True, axis='y', alpha=0.3, linestyle='--')
    ax1.set_axisbelow(True)
    
    total_1 = data_1['market_amount'].sum()
    ax1.text(0.98, 0.97, f'合計: {total_1:,.0f}億円',
             transform=ax1.transAxes, fontsize=10,
             verticalalignment='top', horizontalalignment='right',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # グラフ2: 日付2
    ax2.bar(data_2['maturity_bucket'],
            data_2['market_amount'],
            width=0.8,
            color='coral',
            edgecolor='darkred',
            alpha=0.7)
    
    ax2.set_title(f'年限別市中残存額\n{date_2} 時点',
                  fontsize=14, fontweight='bold', pad=15)
    ax2.set_xlabel('残存期間（年）', fontsize=11)
    ax2.set_ylabel('市中残存額（億円）', fontsize=11)
    ax2.set_xticks(data_2['maturity_bucket'])
    ax2.set_xticklabels(data_2['bucket_label'], rotation=45, ha='right', fontsize=8)
    ax2.grid(True, axis='y', alpha=0.3, linestyle='--')
    ax2.set_axisbelow(True)
    
    total_2 = data_2['market_amount'].sum()
    ax2.text(0.98, 0.97, f'合計: {total_2:,.0f}億円',
             transform=ax2.transAxes, fontsize=10,
             verticalalignment='top', horizontalalignment='right',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # Y軸の範囲を揃える
    max_y = max(data_1['market_amount'].max(), data_2['market_amount'].max())
    ax1.set_ylim(0, max_y * 1.1)
    ax2.set_ylim(0, max_y * 1.1)
    
    plt.tight_layout()
    plt.show()
    
    # 差分の表示
    print('\n【変動サマリー】')
    print(f'{date_1} 合計: {total_1:,.0f}億円')
    print(f'{date_2} 合計: {total_2:,.0f}億円')
    print(f'差分: {total_1 - total_2:+,.0f}億円 ({((total_1 - total_2) / total_2 * 100):+.2f}%)')


# 比較グラフを表示
plot_comparison(data_1, DATE_1, data_2, DATE_2)